In [ ]:
!pip install statsforecast


In [28]:
import os
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from scipy.stats import norm
from statsforecast import StatsForecast
from statsforecast.models import (
    HoltWinters,
    CrostonClassic as Croston,
    HistoricAverage,
    DynamicOptimizedTheta as DOT,
    SeasonalNaive,
    AutoARIMA
)
url = "https://raw.githubusercontent.com/Thomas101Shen/Hw4-fre/main/Metro_Interstate_Traffic_Volume.csv"

df = pd.read_csv(url)
df.drop(columns="holiday", inplace=True)
df.head()
# df.dropna(inplace=True)
# df.head()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.6/134.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.7/196.7 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.7 MB/s eta 0:00:00


,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
0,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545
1,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516
2,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767
3,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026
4,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,4918


In [23]:
df.dtypes

,0
temp,float64
rain_1h,float64
snow_1h,float64
clouds_all,int64
weather_main,object
weather_description,object
date_time,object
traffic_volume,int64


In [24]:
drop_col = ["weather_main", "weather_description"]
df.drop(columns = drop_col, inplace=True)

In [25]:
df.head()

,temp,rain_1h,snow_1h,clouds_all,date_time,traffic_volume
0,288.28,0.0,0.0,40,2012-10-02 09:00:00,5545
1,289.36,0.0,0.0,75,2012-10-02 10:00:00,4516
2,289.58,0.0,0.0,90,2012-10-02 11:00:00,4767
3,290.13,0.0,0.0,90,2012-10-02 12:00:00,5026
4,291.14,0.0,0.0,75,2012-10-02 13:00:00,4918


In [27]:
df_melted = pd.melt(df, id_vars=['date_time'], value_vars=['temp', 'rain_1h', 'snow_1h', 'clouds_all', 'traffic_volume'],
                    var_name='unique_id', value_name='y')
df_melted.head()

,date_time,unique_id,y
0,2012-10-02 09:00:00,temp,288.28
1,2012-10-02 10:00:00,temp,289.36
2,2012-10-02 11:00:00,temp,289.58
3,2012-10-02 12:00:00,temp,290.13
4,2012-10-02 13:00:00,temp,291.14
